In [1]:
import pandas as pd
import numpy as np

In [2]:
# ==========================================
# STEP 0: Load and Prepare "Messy" Data 
# (Adding issues to fulfill assignment requirements)
# ==========================================
df = pd.read_csv('hr_analytics_dummy_dataset.csv')

# 1. Add duplicates
df = pd.concat([df, df.iloc[:15]], ignore_index=True)

# 2. Add missing values
df.loc[0:20, 'Monthly_Salary'] = np.nan
df.loc[10:30, 'Department'] = np.nan

# 3. Add messy text
df.loc[0:10, 'Job_Role'] = '  senior executive  '
df.loc[11:20, 'Job_Role'] = 'manager '

In [3]:
# ==========================================
# PART 2: Quick Audit
# ==========================================
print("--- PART 2: QUICK AUDIT ---")
print(f"Dataset Shape: {df.shape}")
print("\nFirst 5 Rows:")
display(df.head())

print("\nData Info:")
df.info()

print("\nMissing Values Count (Sorted):")
print(df.isna().sum().sort_values(ascending=False))

print(f"\nDuplicate Rows Count: {df.duplicated().sum()}")

--- PART 2: QUICK AUDIT ---
Dataset Shape: (1215, 10)

First 5 Rows:


,Employee_ID,Age,Gender,Department,Job_Role,Years_At_Company,Monthly_Salary,Performance_Rating,Training_Hours,Attrition
0,50001,25,Female,Operations,senior executive,9.9,NaN,2,20,No
1,50002,46,Female,Finance,senior executive,11.3,NaN,3,31,No
2,50003,24,Male,Marketing,senior executive,9.3,NaN,4,4,No
3,50004,40,Male,Marketing,senior executive,19.6,NaN,5,73,No
4,50005,44,Male,IT,senior executive,15.2,NaN,3,61,No



Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1215 entries, 0 to 1214
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Employee_ID         1215 non-null   int64  
 1   Age                 1215 non-null   int64  
 2   Gender              1215 non-null   object 
 3   Department          1194 non-null   object 
 4   Job_Role            1215 non-null   object 
 5   Years_At_Company    1215 non-null   float64
 6   Monthly_Salary      1194 non-null   float64
 7   Performance_Rating  1215 non-null   int64  
 8   Training_Hours      1215 non-null   int64  
 9   Attrition           1215 non-null   object 
dtypes: float64(2), int64(4), object(4)
memory usage: 95.1+ KB

Missing Values Count (Sorted):
Monthly_Salary        21
Department            21
Age                    0
Employee_ID            0
Job_Role               0
Gender                 0
Years_At_Company       0
Performance_Rating     

In [4]:
# PART 3: Cleaning Work (6 Actions)
# ==========================================
print("\n--- PART 3: CLEANING WORK ---")

# 1. Fix Datatype (Mandatory): Convert Employee_ID to string
df['Employee_ID'] = df['Employee_ID'].astype(str)

# 2. Handle Missing Values (Mandatory - Method 1): Median for Numeric
salary_med = df['Monthly_Salary'].median()
df['Monthly_Salary'] = df['Monthly_Salary'].fillna(salary_med)

# 3. Handle Missing Values (Mandatory - Method 2): Constant for Categorical
df['Department'] = df['Department'].fillna('Unknown')

# 4. Clean Text Columns (Mandatory): Strip whitespace and use Title Case
df['Job_Role'] = df['Job_Role'].str.strip().str.title()

# 5. Remove Duplicates (Additional Action)
df = df.drop_duplicates()

# 6. Rename Columns (Additional Action)
df = df.rename(columns={'Years_At_Company': 'Tenure_Years'})

# 7. Drop Irrelevant Columns (Additional Action)
# Dropping Employee_ID as it is not needed for statistical insights
df = df.drop(columns=['Employee_ID'])

print("Missing values after cleaning:")
print(df.isna().sum())


--- PART 3: CLEANING WORK ---
Missing values after cleaning:
Age                   0
Gender                0
Department            0
Job_Role              0
Tenure_Years          0
Monthly_Salary        0
Performance_Rating    0
Training_Hours        0
Attrition             0
dtype: int64


In [5]:
# ==========================================
# PART 4: Create New Columns (3 Columns)
# ==========================================
print("\n--- PART 4: CREATE NEW COLUMNS ---")

# 1. Salary_Level: Conditional Flag (Boolean Logic)
# If salary is greater than median, label 'High', else 'Low'
df['Salary_Level'] = np.where(df['Monthly_Salary'] > df['Monthly_Salary'].median(), 'High', 'Low')

# 2. Age_Group: Using pd.cut()
df['Age_Group'] = pd.cut(df['Age'], bins=[0, 30, 45, 100], labels=['Young', 'Middle-Aged', 'Senior'])

# 3. Performance_Status: Mapping logic
# Creating a simple flag for high performers who stayed
df['High_Performer_Stayed'] = (df['Performance_Rating'] >= 4) & (df['Attrition'] == 'No')

display(df[['Monthly_Salary', 'Salary_Level', 'Age_Group', 'High_Performer_Stayed']].head())


--- PART 4: CREATE NEW COLUMNS ---


,Monthly_Salary,Salary_Level,Age_Group,High_Performer_Stayed
0,87154.5,Low,Young,False
1,87154.5,Low,Senior,False
2,87154.5,Low,Young,True
3,87154.5,Low,Middle-Aged,True
4,87154.5,Low,Middle-Aged,False


In [6]:
# ==========================================
# PART 5: Insights (5 Insights)
# ==========================================
print("\n--- PART 5: INSIGHTS ---")

# 1. groupby().agg()
dept_stats = df.groupby('Department').agg({'Monthly_Salary': 'mean', 'Tenure_Years': 'mean'}).round(2)
print("Insight 1: Average Salary and Tenure by Department:")
display(dept_stats)

# 2. sort_values()
top_salaries = df.sort_values(by='Monthly_Salary', ascending=False).head(3)
print("\nInsight 2: Top 3 highest earning employees belong to:")
display(top_salaries[['Department', 'Job_Role', 'Monthly_Salary']])

# 3. value_counts()
attrition_count = df['Attrition'].value_counts()
print(f"\nInsight 3: Attrition counts (No vs Yes):\n{attrition_count}")

# 4. pivot_table()
pivot = df.pivot_table(values='Performance_Rating', index='Gender', columns='Salary_Level', aggfunc='mean').round(2)
print("\nInsight 4: Average Performance Rating by Gender and Salary Level:")
display(pivot)

# 5. Filtering with Boolean Conditions
high_performers = df[df['High_Performer_Stayed'] == True].shape[0]
print(f"\nInsight 5: There are {high_performers} employees who are high performers and have not left the company.")

# Save the final cleaned data
df.to_csv('hr_analytics_cleaned.csv', index=False)
print("\nCleaned dataset saved as 'hr_analytics_cleaned.csv'")


--- PART 5: INSIGHTS ---
Insight 1: Average Salary and Tenure by Department:


,Monthly_Salary,Tenure_Years
Department,,
Finance,88007.31,11.00
HR,85045.15,10.48
IT,86382.27,10.71
Marketing,81543.72,9.86
Operations,84802.74,10.05
Unknown,86884.69,10.89



Insight 2: Top 3 highest earning employees belong to:


,Department,Job_Role,Monthly_Salary
399,Finance,Senior Executive,149938.0
535,Finance,Team Lead,149711.0
322,IT,Senior Executive,149621.0



Insight 3: Attrition counts (No vs Yes):
Attrition
No     927
Yes    288
Name: count, dtype: int64

Insight 4: Average Performance Rating by Gender and Salary Level:


Salary_Level,High,Low
Gender,,
Female,2.97,3.01
Male,2.97,2.90



Insight 5: There are 353 employees who are high performers and have not left the company.

Cleaned dataset saved as 'hr_analytics_cleaned.csv'


In [7]:
#Export the clean data frame to a new CSV file
topath=r"D:\3rd sem\python\cleaned hr_analytics_dummy_dataset.csv"
df.to_csv(topath,index=False)